In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy

Part Of Speech

---



In [2]:
# Render the dependency parse immediately inside Jupyter:
def display_pos(input_text):
  doc=nlp(input_text)
  displacy.render(doc, style='dep', jupyter=True, options={'distance': 110})

def token_info(input_text):
  doc=nlp(input_text)
  for token in doc:
      print(f'{token.text:{10}} {token.pos_:{7}} {token.dep_:{7}} {spacy.explain(token.dep_)}')

doc=nlp("show me transaction for Megamart retail")
# Render the dependency parse immediately inside Jupyter:
displacy.render(doc, style='dep', jupyter=True, options={'distance': 110})

DATABASE

---



In [3]:
company_names=["Megamart retail","tally solutions pvt. ltd.", "Vestige", "Modicare", "Herbalife","Amulya Herbs Pvt. Ltd.", "Retail 1947 Enterprises Pvt. ltd."]

In [4]:
import pandas as pd

# Create a sample dataframe
data = {'Company': ["Megamart retail","tally solutions pvt. ltd.", "Vestige", "Modicare", "Herbalife","Amulya Herbs Pvt. Ltd.", "Retail 1947 Enterprises Pvt. ltd.","Megamart retail","tally solutions pvt. ltd.","Megamart retail","tally solutions pvt. ltd.","Megamart retail"],
        'Transaction_Date': ['20230105', '20230105', '20230120', '20230216', '20230305', '20230421', '20230505', '20230519', '20230608', '20230623', '20230711','20230713'],
        'Amount': [1000, 2000, 3000, 4000, 5000, 2000, 3000, 4000, 5000,2500,7000,4500]}
df = pd.DataFrame(data)
df

,Company,Transaction_Date,Amount
0,Megamart retail,20230105,1000
1,tally solutions pvt. ltd.,20230105,2000
2,Vestige,20230120,3000
3,Modicare,20230216,4000
4,Herbalife,20230305,5000
5,Amulya Herbs Pvt. Ltd.,20230421,2000
6,Retail 1947 Enterprises Pvt. ltd.,20230505,3000
7,Megamart retail,20230519,4000
8,tally solutions pvt. ltd.,20230608,5000
9,Megamart retail,20230623,2500


In [5]:
filtered_df = df[(df['Company'] == 'Megamart retail') & (df['Transaction_Date'] >= '20230415') & (df['Transaction_Date'] <= '20231207')]
# filtered_df=filtered_df[['Company','Amount']]
filtered_df

,Company,Transaction_Date,Amount
7,Megamart retail,20230519,4000
9,Megamart retail,20230623,2500
11,Megamart retail,20230713,4500


Date Extraction

---



In [6]:
# date
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta  #to add/subtract months from a date

def get_date(date):
  # Convert relative expressions to standard date format
  today_ = datetime.today().date()
  today=today_.strftime("%Y%m%d")
  tomorrow = (today_ + timedelta(days=1)).strftime("%Y%m%d")
  yesterday = (today_ - timedelta(days=1)).strftime("%Y%m%d")

  relative_expressions = {"yesterday": yesterday, "today": str(today), "tomorrow": tomorrow}

  # Store the modified tokens for yesterday,today and tomorrow.
  modified_tokens = ""
  for token in date:
      if token.text.lower() in relative_expressions:
          modified_token = relative_expressions[token.text.lower()]
      else:
          modified_token = token.text
      modified_tokens=modified_tokens+modified_token

  modified_tokens=nlp(modified_tokens)
  for ent in modified_tokens.ents:
    if ent.label_ == "DATE":
        return ent.text

  for ent in date.ents:
      if ent.label_ == "DATE":
          parsed_date = parse(ent.text)
          return parsed_date.strftime("%Y%m%d")
  return 'No date found'

xml voucher creation and show transaction

---



In [7]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def suggest_company(companies):
    # Assuming you have a list of existing company names
    # company_names = ["Company A", "Company B", "Company C"]
    matched_companies=set()

    for company_name in companies:
      for existing_name in company_names:
          ratio = fuzz.ratio(company_name.lower(), existing_name.lower())
          if ratio > 30:  # Customize the threshold as per your requirements
              matched_companies.add(existing_name)

    suggested_companies=[]
    for company in matched_companies:
      suggested_companies.append(company);

    if len(suggested_companies)==0:
      return "NOT_FOUND"
    return process_query(suggested_companies)

def process_query(list):
  print("Do You Mean one of these : ")
  for i, company in enumerate(list):
      print(f"{i+1}. {company}")

  while (True):
    selection = int(input("\nPlease enter the number corresponding to the company name: "))
    if(selection>=1 and selection<=len(list)):
      selected_company = list[selection - 1]
      return selected_company
    else:
      print("invalid selection")

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Tally voucher XML Code

---





In [8]:
#@title
from xml.dom import minidom
def create_voucher_xml(product_info):
    doc = minidom.Document()

    envelope = doc.createElement("ENVELOPE")
    doc.appendChild(envelope)

    header = doc.createElement("HEADER")
    envelope.appendChild(header)

    tally_request = doc.createElement("TALLYREQUEST")
    tally_request.appendChild(doc.createTextNode("Import Data"))
    header.appendChild(tally_request)

    body = doc.createElement("BODY")
    envelope.appendChild(body)

    import_data = doc.createElement("IMPORTDATA")
    body.appendChild(import_data)

    request_desc = doc.createElement("REQUESTDESC")
    import_data.appendChild(request_desc)

    report_name = doc.createElement("REPORTNAME")
    report_name.appendChild(doc.createTextNode("Vouchers"))
    request_desc.appendChild(report_name)

    static_variables = doc.createElement("STATICVARIABLES")
    request_desc.appendChild(static_variables)

    sv_current_company = doc.createElement("SVCURRENTCOMPANY")
    sv_current_company.appendChild(doc.createTextNode(product_info['company_name']))
    static_variables.appendChild(sv_current_company)

    request_data = doc.createElement("REQUESTDATA")
    import_data.appendChild(request_data)

    tally_message = doc.createElement("TALLYMESSAGE")
    request_data.appendChild(tally_message)

    # for product_info in sales_info:
    voucher = doc.createElement("VOUCHER")
    voucher.setAttribute("VCHTYPE", "Payment") #product_info['voucher_type']
    voucher.setAttribute("ACTION", "Create")
    voucher.setAttribute("OBJVIEW", "Accounting Voucher View")
    tally_message.appendChild(voucher)

    voucher_date = doc.createElement("DATE")
    voucher_date.appendChild(doc.createTextNode(product_info['date']))
    voucher.appendChild(voucher_date)

    voucher_guid = doc.createElement("GUID")
    voucher_guid.appendChild(doc.createTextNode(product_info['guid']))
    voucher.appendChild(voucher_guid)

    voucher_type_name = doc.createElement("VOUCHERTYPENAME")
    voucher_type_name.appendChild(doc.createTextNode('Payment')) #product_info['voucher_type']
    voucher.appendChild(voucher_type_name)

    voucher_number = doc.createElement("VOUCHERNUMBER")
    voucher_number.appendChild(doc.createTextNode(str(product_info['voucher_number'])))
    voucher.appendChild(voucher_number)

    # Additional elements...
    cst_form_issue_type = doc.createElement("CSTFORMISSUETYPE")
    voucher.appendChild(cst_form_issue_type)

    cst_form_recv_type = doc.createElement("CSTFORMRECVTYPE")
    voucher.appendChild(cst_form_recv_type)

    persisted_view = doc.createElement("PERSISTEDVIEW")
    persisted_view.appendChild(doc.createTextNode("Accounting Voucher View"))
    voucher.appendChild(persisted_view)

    vch_gst_class = doc.createElement("VCHGSTCLASS")
    voucher.appendChild(vch_gst_class)

    diff_actual_qty = doc.createElement("DIFFACTUALQTY")
    diff_actual_qty.appendChild(doc.createTextNode("No"))
    voucher.appendChild(diff_actual_qty)

    is_deleted = doc.createElement("ISDELETED")
    is_deleted.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_deleted)

    is_original = doc.createElement("ASORIGINAL")
    is_original.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_original)

    for_job_costing = doc.createElement("FORJOBCOSTING")
    for_job_costing.appendChild(doc.createTextNode("No"))
    voucher.appendChild(for_job_costing)

    is_optional = doc.createElement("ISOPTIONAL")
    is_optional.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_optional)

    effective_date = doc.createElement("EFFECTIVEDATE")
    effective_date.appendChild(doc.createTextNode(product_info['date']))
    voucher.appendChild(effective_date)

    date=doc.createElement("DATE")
    date.appendChild(doc.createTextNode(product_info['date']))
    voucher.appendChild(date)

    use_for_excise = doc.createElement("USEFOREXCISE")
    use_for_excise.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_for_excise)

    use_for_interest = doc.createElement("USEFORINTEREST")
    use_for_interest.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_for_interest)

    use_for_gain_loss = doc.createElement("USEFORGAINLOSS")
    use_for_gain_loss.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_for_gain_loss)

    use_for_godown_transfer = doc.createElement("USEFORGODOWNTRANSFER")
    use_for_godown_transfer.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_for_godown_transfer)

    use_for_compound = doc.createElement("USEFORCOMPOUND")
    use_for_compound.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_for_compound)

    use_for_service_tax = doc.createElement("USEFORSERVICETAX")
    use_for_service_tax.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_for_service_tax)

    is_on_hold = doc.createElement("ISONHOLD")
    is_on_hold.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_on_hold)

    excise_tax_override = doc.createElement("EXCISETAXOVERRIDE")
    excise_tax_override.appendChild(doc.createTextNode("No"))
    voucher.appendChild(excise_tax_override)

    ignore_pos_validation = doc.createElement("IGNOREPOSVALIDATION")
    ignore_pos_validation.appendChild(doc.createTextNode("No"))
    voucher.appendChild(ignore_pos_validation)

    is_tds_overridden = doc.createElement("ISTDSOVERRIDDEN")
    is_tds_overridden.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_tds_overridden)

    is_tcs_overridden = doc.createElement("ISTCSOVERRIDDEN")
    is_tcs_overridden.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_tcs_overridden)

    is_vat_overridden = doc.createElement("ISVATOVERRIDDEN")
    is_vat_overridden.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_vat_overridden)

    is_service_tax_overridden = doc.createElement("ISSERVICETAXOVERRIDDEN")
    is_service_tax_overridden.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_service_tax_overridden)

    is_excise_overridden = doc.createElement("ISEXCISEOVERRIDDEN")
    is_excise_overridden.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_excise_overridden)

    is_gst_overridden = doc.createElement("ISGSTOVERRIDDEN")
    is_gst_overridden.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_gst_overridden)

    ignore_gst_invalidation = doc.createElement("IGNOREGSTINVALIDATION")
    ignore_gst_invalidation.appendChild(doc.createTextNode("No"))
    voucher.appendChild(ignore_gst_invalidation)

    ignore_einv_validation = doc.createElement("IGNOREEINVVALIDATION")
    ignore_einv_validation.appendChild(doc.createTextNode("No"))
    voucher.appendChild(ignore_einv_validation)

    is_cancelled = doc.createElement("ISCANCELLED")
    is_cancelled.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_cancelled)

    has_cashflow = doc.createElement("HASCASHFLOW")
    has_cashflow.appendChild(doc.createTextNode("No"))
    voucher.appendChild(has_cashflow)

    is_post_dated = doc.createElement("ISPOSTDATED")
    is_post_dated.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_post_dated)

    use_tracking_number = doc.createElement("USETRACKINGNUMBER")
    use_tracking_number.appendChild(doc.createTextNode("No"))
    voucher.appendChild(use_tracking_number)

    is_invoice = doc.createElement("ISINVOICE")
    is_invoice.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_invoice)

    mfg_journal = doc.createElement("MFGJOURNAL")
    mfg_journal.appendChild(doc.createTextNode("No"))
    voucher.appendChild(mfg_journal)

    has_discounts = doc.createElement("HASDISCOUNTS")
    has_discounts.appendChild(doc.createTextNode("No"))
    voucher.appendChild(has_discounts)

    as_payslip = doc.createElement("ASPAYSLIP")
    as_payslip.appendChild(doc.createTextNode("No"))
    voucher.appendChild(as_payslip)

    is_cost_centre = doc.createElement("ISCOSTCENTRE")
    is_cost_centre.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_cost_centre)

    is_stx_non_realized_vch = doc.createElement("ISSTXNONREALIZEDVCH")
    is_stx_non_realized_vch.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_stx_non_realized_vch)

    is_blank_cheque = doc.createElement("ISBLANKCHEQUE")
    is_blank_cheque.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_blank_cheque)

    is_void = doc.createElement("ISVOID")
    is_void.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_void)

    order_line_status = doc.createElement("ORDERLINESTATUS")
    order_line_status.appendChild(doc.createTextNode("No"))
    voucher.appendChild(order_line_status)

    is_vat_duty_paid = doc.createElement("ISVATDUTYPAID")
    is_vat_duty_paid.appendChild(doc.createTextNode("Yes"))
    voucher.appendChild(is_vat_duty_paid)

    is_deleted_vch_retained = doc.createElement("ISDELETEDVCHRETAINED")
    is_deleted_vch_retained.appendChild(doc.createTextNode("No"))
    voucher.appendChild(is_deleted_vch_retained)

    curr_party_ledger_name = doc.createElement("CURRPARTYLEDGERNAME")
    voucher.appendChild(curr_party_ledger_name)

    ewaybill_details_list = doc.createElement("EWAYBILLDETAILS.LIST")
    voucher.appendChild(ewaybill_details_list)

    excluded_taxations_list = doc.createElement("EXCLUDEDTAXATIONS.LIST")
    voucher.appendChild(excluded_taxations_list)

    old_audit_entries_list = doc.createElement("OLDAUDITENTRIES.LIST")
    voucher.appendChild(old_audit_entries_list)

    account_audit_entries_list = doc.createElement("ACCOUNTAUDITENTRIES.LIST")
    voucher.appendChild(account_audit_entries_list)

    audit_entries_list = doc.createElement("AUDITENTRIES.LIST")
    voucher.appendChild(audit_entries_list)

    duty_head_details_list = doc.createElement("DUTYHEADDETAILS.LIST")
    voucher.appendChild(duty_head_details_list)

    supplementary_duty_head_details_list = doc.createElement("SUPPLEMENTARYDUTYHEADDETAILS.LIST")
    voucher.appendChild(supplementary_duty_head_details_list)

    ewaybill_error_list = doc.createElement("EWAYBILLERRORLIST.LIST")
    voucher.appendChild(ewaybill_error_list)

    irn_error_list = doc.createElement("IRNERRORLIST.LIST")
    voucher.appendChild(irn_error_list)

    invoice_del_notes_list = doc.createElement("INVOICEDELNOTES.LIST")
    voucher.appendChild(invoice_del_notes_list)

    invoice_order_list = doc.createElement("INVOICEORDERLIST.LIST")
    voucher.appendChild(invoice_order_list)

    invoice_indent_list = doc.createElement("INVOICEINDENTLIST.LIST")
    voucher.appendChild(invoice_indent_list)

    attendance_entries_list = doc.createElement("ATTENDANCEENTRIES.LIST")
    voucher.appendChild(attendance_entries_list)

    origin_invoice_details_list = doc.createElement("ORIGINVOICEDETAILS.LIST")
    voucher.appendChild(origin_invoice_details_list)

    invoice_export_list = doc.createElement("INVOICEEXPORTLIST.LIST")
    voucher.appendChild(invoice_export_list)

    oth_ref_details_list = doc.createElement("OTHREFDETAILS.LIST")
    voucher.appendChild(oth_ref_details_list)

    extra_bill_allocation_list = doc.createElement("EXTRABILLALLOCATIONLIST.LIST")
    voucher.appendChild(extra_bill_allocation_list)

    fbtp_details_list = doc.createElement("FBTPDETAILS.LIST")
    voucher.appendChild(fbtp_details_list)

    item_nominal_allocation_list = doc.createElement("ITEMNOMINALALLOCATION.LIST")
    voucher.appendChild(item_nominal_allocation_list)

    ref_invoice_details_list = doc.createElement("REFINVOICEDETAILS.LIST")
    voucher.appendChild(ref_invoice_details_list)

    inward_details_list = doc.createElement("INWARDDETAILS.LIST")
    voucher.appendChild(inward_details_list)

    outward_supply_details_list = doc.createElement("OUTWARDSUPPLYDETAILS.LIST")
    voucher.appendChild(outward_supply_details_list)

    voucher_foreign_details_list = doc.createElement("VOUCHERFOREIGNDETAILS.LIST")
    voucher.appendChild(voucher_foreign_details_list)

    lcv_details_list = doc.createElement("LCVDETAILS.LIST")
    voucher.appendChild(lcv_details_list)

    abatement_details_list = doc.createElement("ABATEMENTDETAILS.LIST")
    voucher.appendChild(abatement_details_list)

    stax_details_list = doc.createElement("STAXDETAILS.LIST")
    voucher.appendChild(stax_details_list)

    vat_classifications_list = doc.createElement("VATCLASSIFICATIONS.LIST")
    voucher.appendChild(vat_classifications_list)

    invoice_item_excise_details_list = doc.createElement("INVOICEITEMEXCISEDETAILS.LIST")
    voucher.appendChild(invoice_item_excise_details_list)

    invoice_item_details_list = doc.createElement("INVOICEITEMDETAILS.LIST")
    voucher.appendChild(invoice_item_details_list)

    invoice_tax_classifications_list = doc.createElement("INVOICETAXCLASSIFICATIONS.LIST")
    voucher.appendChild(invoice_tax_classifications_list)

    invoice_sub_category_details_list = doc.createElement("INVOICESUBCATEGORYDETAILS.LIST")
    voucher.appendChild(invoice_sub_category_details_list)

    bill_details_list = doc.createElement("BILLDETAILS.LIST")
    voucher.appendChild(bill_details_list)

    ref_voucher_detail_list = doc.createElement("REFVOUCHERDETAIL.LIST")
    voucher.appendChild(ref_voucher_detail_list)

    voucher_tax_details_list = doc.createElement("VOUCHERTAXDETAILS.LIST")
    voucher.appendChild(voucher_tax_details_list)

    voucher_extra_tally_lic_details_list = doc.createElement("VOUCHEREXTRATALLYLICDETAILS.LIST")
    voucher.appendChild(voucher_extra_tally_lic_details_list)

    with open("sales_voucher.xml", "w") as f:
        f.write(doc.toprettyxml(indent="  "))

    print("XML file created successfully.")

In [9]:
def voucher_creation():
  print(f'Creating {task} for {company}')
  # dealing with date
  date=input('\ncan you please tell me about date of sale ? ')
  date=get_date(nlp(date))
  while date=='No date found':
    date=input('No date found.Please enter the date again.')
    date=get_date(nlp(date))

  while True:
    doc=input('\nwhat is your transaction amount ? \n')
    amount=get_amount(doc)
    if amount!='no amount':
      trans_amount=amount
      break

  product_info={
      'voucher_type': "voucher_type",
      'date': date,
      'voucher_number': "1",
      'company_name': company,#company_name,
      'ledger_name': "ledger_name",
      'amount': "amount",
      'guid': "guid"
  }

  #Adding new voucher data to my dataframe

  # new_row = {'Company': company, 'Transaction_Date': date, 'Amount': trans_amount}
  # df = df.append(new_row, ignore_index=True)
  create_voucher_xml(product_info)

def show_transaction(task,company):
  print("\n"f'sure! let me show {task} for {company}')
  types=['weekly','monthly','quarterly','semi-annual','annual','give a range of date']

  print("What Report you want to see ? : ")
  for i, type in enumerate(types):
      print(f"{i+1}. {type}")

  type=""
  while (True):
    selection = int(input("\nEnter number corresponding to type : "))
    if(selection>=1 and selection<=len(types)):
      type = types[selection - 1]
      break
    else:
      print("invalid selection")

  current=""
  prev=""
  today_ = datetime.today().date()
  today=today_.strftime("%Y%m%d")

  # print(df)

  if type=='give a range of date':
    prev=get_date(nlp(input('from date : ')))
    current=get_date(nlp(input('to date : ')))
  else:
    current=today
    if type=='weekly':
      prev = (today_ - timedelta(days=6)).strftime("%Y%m%d")
    elif type=='monthly':
      prev = (today_ - timedelta(days=29)).strftime("%Y%m%d")
    elif type=='quarterly':
      prev=(today_ - relativedelta(months=4)).strftime("%Y%m%d")
    elif type=='semi-annual':
      prev=(today_ - relativedelta(months=6)).strftime("%Y%m%d")
    elif type=='annual':
      prev=(today_ - relativedelta(months=12)).strftime("%Y%m%d")

  filtered_df = df[(df['Company'] == company) & (df['Transaction_Date'] >= prev) & (df['Transaction_Date'] <= '20231207')]
  print(filtered_df)

def get_amount(doc):
  doc=nlp(doc)
  for token in doc:
    if token.pos_ == 'NUM':
      return token.text

  return 'no amount'

In [10]:
query=""
task=""
adj=""
companies=[]
trans_amount=""

input_text=input("How can i help you ? \n")
doc=nlp(input_text)
for token in doc:
  if token.pos_== 'VERB' and query=="":
    query=(token.text)
  elif token.pos_ == 'NOUN' and task=="":
    task=(token.text)
  elif token.pos_ == 'NOUN' and task!="":
    companies.append(token.text)
  elif token.pos_ == 'ADJ' and adj=="":
    adj=(token.text)
  elif token.pos_ == 'PROPN':
    companies.append(token.text)

company=suggest_company(companies)
if query.lower()=='create' and task.lower()=='voucher' :
  voucher_creation()
elif query.lower()== 'show' and task=='transaction':
  show_transaction(task,company)

How can i help you ? 
create me a voucher for megamart
Do You Mean one of these : 
1. Megamart retail
2. Modicare
3. Amulya Herbs Pvt. Ltd.

Please enter the number corresponding to the company name: 1
Creating voucher for Megamart retail

can you please tell me about date of sale ? today

what is your transaction amount ? 
5000
XML file created successfully.
